In [1]:
import torch
import pandas as pd
import numpy as np

In [2]:
# ID processing
ids =  open("interpro_term_ids.txt", "r")
lines = ids.read().split()
id_dict = {}
for i in range(0,len(lines),2):
    # print(lines[i])
    # print(lines[i+1])
    id_dict[i/2] = lines[i]
    tes

In [3]:
# Term Processing 
terms = open("interpro_terms.txt", "r")
lines = terms.read().split('\n')
term_dict = {}
num_categories = 1
saved = ""
for i in range(len(lines)):
    line_array = lines[i].split()
    term_dict[line_array[0]] = list(map(int, line_array[1:]))
    # if num_categories < max(term_dict[line_array[0]]):
    #     saved = line_array
    num_categories = max(num_categories, max(term_dict[line_array[0]]))
    
#     if i > 10:
#         break
# print(term_dict)
num_categories = num_categories+1
print(num_categories)
# print(saved)


27612


In [4]:
multi_hot_labels = torch.zeros((len(term_dict),num_categories))
count = 0
for prot in term_dict:
    # temp = torch.zeros(num_categories)
    int_terms = term_dict[prot]
    # temp[int_terms] = 1
    multi_hot_labels[count][int_terms] = 1
    # multi_hot_labels = torch.cat((multi_hot_labels,temp),1)
    count += 1
print(multi_hot_labels.shape)
print(torch.sum(multi_hot_labels))


torch.Size([127218, 27612])
tensor(541716.)


In [38]:
from sklearn.decomposition import NMF, PCA, TruncatedSVD
# model = NMF(n_components=1024, init= 'random', random_state=0)
# W = model.fit_transform(multi_hot_labels)
svd = TruncatedSVD(n_components=1024)
svd.fit(multi_hot_labels)

In [18]:
from sklearn.decomposition import NMF, PCA, TruncatedSVD
# model = NMF(n_components=1024, init= 'random', random_state=0)
# W = model.fit_transform(multi_hot_labels)
svd_trans = TruncatedSVD(n_components=2)
svd_trans.fit(torch.transpose(multi_hot_labels,0,1))

TruncatedSVD()

In [24]:
left_sing_vect = np.matmul(multi_hot_labels,np.transpose(svd.components_))


In [37]:
# print(np.shape(svd.components_))
# print(np.shape(left_sing_vect))
# print(np.shape(svd_trans.components_))
# print(np.shape(svd.transform(multi_hot_labels)))
# print(left_sing_vect)
trans_Int = svd.transform(multi_hot_labels)
np.save("Transformed_Interpro",trans_Int)
# print(svd.n_features_in_)

[[ 4.4540856e-03  1.3745555e-02]
 [ 7.6809247e-06  3.2748242e-06]
 [ 4.2494116e-06  4.4854132e-06]
 ...
 [ 1.8362008e-03  1.0325168e-03]
 [ 9.5295710e-22  2.8189123e-20]
 [ 2.2389402e-04 -5.9745670e-04]]
tensor([[ 4.4541e-03,  1.3746e-02],
        [ 7.6809e-06,  3.2748e-06],
        [ 4.2494e-06,  4.4854e-06],
        ...,
        [ 1.8362e-03,  1.0325e-03],
        [ 9.5296e-22,  2.8189e-20],
        [ 2.2389e-04, -5.9746e-04]])


In [35]:
# Code for getting homology for randomly sampled proteins from train dataset, 
import requests
import random
#randomly sampling train uniprot ids
random.seed(0)
sampled_protein_indicies = random.sample(range(0,len(term_dict)),1)
sampled_uniprot = []
# response = requests.get("https://string-db.org/api/tsv/homology?identifiers=[your_identifiers]")
# https://string-db.org/api/tsv/homology?identifiers=CDK1%0dCDK2
responses = []
api_str = "https://string-db.org/api/tsv/homology?identifiers=" 
for i in range(len(sampled_protein_indicies)):
    cur_uniprot = [*term_dict][sampled_protein_indicies[i]]
    sampled_uniprot.append(cur_uniprot)  
    response = requests.get(api_str + cur_uniprot)
    responses.append(response)
# help(response)
print(response.iter_content)

<bound method Response.iter_content of <Response [200]>>


In [31]:
lit = term_dict
print([*lit][1])

<>:2: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
<>:2: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
C:\Users\bchiu\AppData\Local\Temp\ipykernel_21728\3177927013.py:2: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  print([*lit][1,3])
C:\Users\bchiu\AppData\Local\Temp\ipykernel_21728\3177927013.py:2: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  print([*lit][1,3])
C:\Users\bchiu\AppData\Local\Temp\ipykernel_21728\3177927013.py:2: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  print([*lit][1,3])


TypeError: list indices must be integers or slices, not tuple

[82333, 58542, 70545, 42445, 48130, 49734, 100951, 44419, 59107, 112221]
